<a href="https://colab.research.google.com/github/arturossouza/colab-test/blob/main/M%C3%A9todos_Matem%C3%A1ticos_Trabalho_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import pandas as pd

In [30]:
!git commit -m "importing pandas"

On branch main
nothing to commit, working tree clean
